# Photometry reduction and lightcurve plot

In [ ]:
__author__ = 'Pascal Louis <plouis35@gmail.com>'
__version__ = '0.1 alpha'
__keywords__ = ['astronomy', 'astropy', 'ccdproc', 'photutils', 'matplotlib']
__forked_from__ = 'https://hebl.china-vo.org/course/PIA2020/et_schmidt_datareduction.pdf'

# Configuration

In [ ]:
### local configuration

raw_data = '../../raw_data/'
reduced_data = '../../reduced_data/'
bias = 'd*BIASR*.fit'      # zero (bias) full frames with overscan subtracted (size: 4096x4096)
flats = 'd*FLATR*.fit'     # dome flat full frames with overscan subtracted
sciences = 'd*UW32R*.fit'  # truncated science frames (an subregion with size of 512x512


In [ ]:
#%matplotlib widget
import warnings
from astropy.utils.exceptions import AstropyWarning
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
import glob

warnings.simplefilter('ignore', category=AstropyWarning)
warnings.simplefilter('ignore', UserWarning)


# Reduction

## Check images and data

In [ ]:
# example of the FITS header and check the data

head=fits.getheader(raw_data + 'd4466637UW32R008.fit')
data=fits.getdata(raw_data + "d4466637UW32R004.fit")
print(head)
f,axs=plt.subplots(1,2,figsize=(10,8))
axs[0].imshow(data,vmin=300,vmax=600,origin='lower')
axs[0].set_title("north down, east left")
axs[1].imshow(np.flipud(data),vmin=300,vmax=600,origin='lower')
axs[1].set_title("north up, east left")
plt.tight_layout()

## Create masterbias

In [ ]:
## create the masterbias image

bfiles=glob.glob(raw_data + bias)
bfiles.sort()
allbias=[]
print("combining bias ...")
for i,ifile in enumerate(bfiles):
    print("reading bias:", i+1,len(bfiles),ifile)
    data=fits.getdata(ifile)
    allbias.append(data)
allbias=np.stack(allbias)
print(allbias.shape)
superbias=np.median(allbias,axis=0)
fits.writeto(reduced_data + 'bias.fit',superbias.astype('float32'),overwrite=True)
print (superbias.min(), superbias.max(), superbias.mean(), superbias.std())

In [ ]:
## display the masterbias
plt.figure(figsize=(8,8))
plt.imshow(superbias,vmin=-5,vmax=5,origin='lower')
plt.colorbar()
plt.title("super bias derived from bias frames")

## Create masterflat

In [ ]:
ffiles=glob.glob(raw_data + flats)
ffiles.sort()
allflat=[]
print("combining dome flats...")
for i,ifile in enumerate(ffiles):
    print("reading flat:", i+1,len(ffiles),ifile)
    # flat-fielding: subtract bias and then normalize the flat images
    data=fits.getdata(ifile)-superbias
    mflat=np.median(data[1500-256:1500+256,1500-256:1500+256])
    data/=mflat
    #print("median flat:",mflat)
    allflat.append(data)
allflat=np.stack(allflat)
print(allflat.shape)
domeflat=np.median(allflat,axis=0)
fits.writeto(reduced_data + 'domeflat.fit',domeflat.astype('float32'),overwrite=True)
print (domeflat.min(), domeflat.max(), domeflat.mean(), domeflat.std())


In [ ]:
## display the masterflat
plt.figure(figsize=(8,8))
plt.imshow(domeflat,origin='lower')
plt.colorbar()
plt.title("dome flat derived from dome flats")

## Calculate gain and readout noise

In [ ]:
## calculate gain and read noise
from astropy.stats import sigma_clipped_stats
biasfile1=raw_data + 'd4466637BIASR213.fit'
biasfile2=raw_data + 'd4466637BIASR214.fit'
flatfile1=raw_data + 'd4466637FLATR201.fit'
flatfile2=raw_data + 'd4466637FLATR202.fit'
bias1=fits.getdata(biasfile1)[1500-256:1500+256,1500-256:1500+256]
bias2=fits.getdata(biasfile2)[1500-256:1500+256,1500-256:1500+256]
flat1=fits.getdata(flatfile1)[1500-256:1500+256,1500-256:1500+256]
flat2=fits.getdata(flatfile2)[1500-256:1500+256,1500-256:1500+256]
mean_flat1=np.median(flat1)
mean_flat2=np.median(flat2)
mean_bias1=np.median(bias1)
mean_bias2=np.median(bias2)
_,_,std_biasdiff=sigma_clipped_stats(bias1-bias2,sigma=4.0,maxiters=2)
_,_,std_flatdiff=sigma_clipped_stats(flat1-flat2,sigma=4.0,maxiters=2)
#print(mean_bias1,mean_bias2,mean_flat1,mean_flat2,std_biasdiff,std_flatdiff)
gain=((mean_flat1+mean_flat2)-(mean_bias1+mean_bias2))/((std_flatdiff**2-std_biasdiff**2))
rdnoise=gain*std_biasdiff/np.sqrt(2)
print("gain: ",gain, "readout noise:",rdnoise)

## Get object coord

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
import glob,os
c=SkyCoord("02:04:10.278","+46:41:16.2",frame='icrs',unit=(u.hourangle,u.degree))
ra=c.ra.degree; dec=c.dec.degree
print("ra:", ra, "dec:",c.dec.degree)
epoch=2000.0 
pixscale=0.137 # in arcsec


## Reduce sciences frames

In [ ]:
## get the center of FoV (pointing to the target) and adding parameters to the FITS header
# add keywords to FITS header of science frames
outdir=reduced_data
sfiles=glob.glob(raw_data + sciences) # science frames
sfiles.sort() # in alphabetic order 
for i,ifile in enumerate(sfiles):
    #print("reducing (debias, flat-fielding, and flipping) :", i+1,len(sfiles),ifile)
    indir,infile=os.path.split(ifile)
    rootname,_=os.path.splitext(infile)
    # we change the first character from "d" to "p" for new files and ensure not to cover the raw data
    outfile=os.path.join(outdir,"p"+rootname[1:]+'.fit') 
    head=fits.getheader(ifile,output_verifystr="silentfix")
    # get the origin of the subregion
    col_origin=head['CRVAL1']
    row_origin=head['CRVAL2']
    subflat=domeflat[row_origin:row_origin+512,col_origin:col_origin+512]
    subbias=superbias[row_origin:row_origin+512,col_origin:col_origin+512]
    if i==0:
        ## to show an example of the subsection of bias and flat
        fits.writeto('../../reduced_data/subflat.fit',subflat.astype('float32'),overwrite=True)
        fits.writeto('../../reduced_data/subbias.fit',subbias.astype('float32'),overwrite=True)
        #break
    data=fits.getdata(ifile)
    
    # de-bias and flat-fielding
    data=(data-subbias)/subflat
    # set the initial reference point in the WCS parameters if doing astrometry 
    head['epoch']=2000.0
    head['CRVAL1']=ra
    head['CRVAL2']=dec
    head['CRPIX1']=head['NAXIS1']/2.0
    head['CRPIX2']=head['NAXIS2']/2.0
    head['CDELT1']=-pixscale/3600.0 # minus for left east
    head['CDELT2']=pixscale/3600.0
    head['CTYPE1']='RA---TAN' # projection type
    head['CTYPE2']='DEC--TAN'
    head['GAIN']=(gain,'GAIN in e-/ADU')
    head['RDNOISE']=(rdnoise,'readout noise in electron')
    print("writing to "+outfile)
    # flip up down to make the image with north up and east left
    fits.writeto(outfile,np.flipud(data),header=head,overwrite=True,output_verify="silentfix")

# Photometry
- Compute relative position shift between different exposures and do aperture photometry using the python package of photutils (no astrometry done)

## Generate sources mask

In [ ]:
from astropy.io import fits
import photutils as pht
from astropy.stats import sigma_clipped_stats, SigmaClip
from photutils.segmentation import detect_threshold, detect_sources
from photutils.utils import circular_footprint

data=fits.getdata(reduced_data + 'p4466637UW32R004.fit')
mean, median, std = sigma_clipped_stats(data, sigma=3.0)
print((mean, median, std))  

sigma_clip = SigmaClip(sigma=3.0, maxiters=10)
threshold = detect_threshold(data, nsigma=2.0, sigma_clip=sigma_clip)
segment_img = detect_sources(data, threshold, npixels=5)
footprint = circular_footprint(radius=3)
mask = segment_img.make_source_mask(footprint=footprint)

mean, median, std = sigma_clipped_stats(data, sigma=3.0, mask=mask)
print((mean, median, std))  

f,axs=plt.subplots(1,2,figsize=(16,8))
axs[0].imshow(data,vmin=300,vmax=600,origin='lower')
axs[0].set_title("data")
axs[1].imshow(mask,origin='lower')
axs[1].set_title("mask");

## Generate background sky

In [ ]:
## get 2D sky map
from astropy.stats import SigmaClip
from photutils.background import SExtractorBackground
from photutils.background import Background2D
sigma_clip = SigmaClip(sigma=3.)
bkg_estimator = SExtractorBackground()
bkg = Background2D(data, (64, 64), mask=mask,filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)

In [ ]:
print(bkg.background_median,bkg.background_rms_median)

In [ ]:
f,axs=plt.subplots(1,2,figsize=(16,8))
axs[0].imshow(bkg.background,origin='lower')
axs[0].set_title("background")
axs[1].imshow(bkg.background_rms,origin='lower')
axs[1].set_title("background rms")

## Detect sources

In [ ]:
## find objects and calculate basic information
from photutils.detection import IRAFStarFinder, DAOStarFinder

daofind = IRAFStarFinder(fwhm=3.0, 
                         threshold=5.*bkg.background_rms_median,
                         exclude_border=True,
                         sharplo=0.5, 
                         sharphi=2.0, 
                         roundlo=0.0, 
                         roundhi=0.7)
"""
daofind = DAOStarFinder(fwhm=1.0, 
                         threshold=3.*bkg.background_rms_median,
                         exclude_border=True,
                         sharplo=0.5, 
                         sharphi=2.0, 
                         roundlo=0.0, 
                         roundhi=0.7)
"""
sources = daofind(data - bkg.background_median)


In [ ]:
print(sources)
#print(sources.colnames)

## Show sources

In [ ]:
#from photutils import CircularAperture
from photutils.aperture import CircularAperture
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=10.)
plt.figure(figsize=(10,8))
plt.imshow(data, cmap='Greys_r', origin='lower', vmin=300,vmax=600, interpolation='nearest')
apertures.plot(color='red', lw=1.5, alpha=0.5);


In [ ]:
## error estimation = photon poisson error + background error
from photutils.utils import calc_total_error
#gain = 2.
error=calc_total_error(data-bkg.background, bkg.background_rms, gain)
print(np.median(error))

## Check photometry (aperture mode)

In [ ]:
## aperture photometry
from astropy.table import Table
from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry
from astropy import table

radii=[3,4,5,6,8,10,12] ## aperture radii in pixels
positions=[(ix,iy) for ix,iy in zip(sources['xcentroid'],sources['ycentroid'])]
apertures = [CircularAperture(positions, r=r) for r in radii]
aper_phot= aperture_photometry(data - bkg.background, apertures, error=error)
#convert flux to magnitude, using a instrumental zeropoint of 25
for i in range(len(radii)):
    fcol='aperture_sum_'+str(i)
    ecol='aperture_sum_err_'+str(i)
    flux=aper_phot[fcol]
    fluxerr=aper_phot[ecol]
    mag=-2.5*np.log10(flux)+25
    magerr=2.5/(flux*np.log(10))*fluxerr
    aper_phot[fcol]=mag
    aper_phot[ecol]=magerr
    aper_phot.rename_column(fcol,'mag_'+str(i))
    aper_phot.rename_column(ecol,'magerr_'+str(i))

In [ ]:
#print(aper_phot.colnames)
print(aper_phot)
#print(aper_phot['mag_8'], aper_phot['magerr_8'])

## Process sciences frames

In [ ]:
from photutils.background import SExtractorBackground
from photutils.background import Background2D
from photutils.detection import IRAFStarFinder
from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry

cfiles=glob.glob(reduced_data + "p*UW32R*.fit") # science frames
cfiles.sort() # in alphabetic order
radii=[3,4,5,6,8,10,12,15] ## aperture radii in pixels
for i,ifile in enumerate(cfiles):
    print("aperture photometry :", i+1,len(cfiles),ifile)
    rootname,_=os.path.splitext(ifile)
    catfile=rootname+'-cat.fits'
    data=fits.getdata(ifile)
    ## or first mask sources then estimate the sky background

    sigma_clip = SigmaClip(sigma=3.0, maxiters=10)
    threshold = detect_threshold(data, nsigma=2.0, sigma_clip=sigma_clip)
    segment_img = detect_sources(data, threshold, npixels=5)
    footprint = circular_footprint(radius=5)
    mask = segment_img.make_source_mask(footprint=footprint)
    mean, median, std = sigma_clipped_stats(data, sigma=3.0, mask=mask)
    
    bkg_estimator = SExtractorBackground()
    bkg = Background2D(data, (64, 64), mask=mask,filter_size=(3, 3), 
                           sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    #print(bkg.background_median,bkg.background_rms_median)
    daofind = IRAFStarFinder(fwhm=3.0, threshold=5.*bkg.background_rms_median,
                                 exclude_border=True, sharplo=0.5, sharphi=2.0, roundlo=0.0, roundhi=0.7)
    sources = daofind(data - bkg.background)
    positions=[(ix,iy) for ix,iy in zip(sources['xcentroid'],sources['ycentroid'])]
    apertures = [CircularAperture(positions, r=r) for r in radii]
    error=calc_total_error(data-bkg.background, bkg.background_rms, gain)
    aper_phot= aperture_photometry(data - bkg.background, apertures, error=error)
    #print(len(aper_phot))
    #convert flux to magnitude, using a instrumental zeropoint of 25
    #for j in range(len(radii)):
    # fcol='aperture_sum_'+str(j)
    # ecol='aperture_sum_err_'+str(j)
    #flux=aper_phot[fcol]
    #fluxerr=aper_phot[ecol]
    #mag=-2.5*np.log10(flux)+25
    #magerr=2.5/(flux*np.log(10))*fluxerr
    #aper_phot[fcol]=mag
    #aper_phot[ecol]=magerr
    #aper_phot.rename_column(fcol,'mag_'+str(j))
    #aper_phot.rename_column(ecol,'magerr_'+str(j))
    aper_phot.write(catfile,overwrite=True)


## Check shift

In [ ]:
## display two images to see the shift
f,axs=plt.subplots(1,2,figsize=(10,8))
data1=fits.getdata(reduced_data + 'p4466637UW32R004.fit')
data2=fits.getdata(reduced_data + 'p4466637UW32R007.fit')
axs[0].imshow(data1,vmin=300,vmax=600,origin='lower')
axs[0].set_title("image 1")
axs[1].imshow(data2,vmin=300,vmax=600,origin='lower')
axs[1].set_title("image 2")

In [ ]:
cat1 = Table.read(reduced_data + 'p4466637UW32R004-cat.fits')
cat2 = Table.read(reduced_data + 'p4466637UW32R007-cat.fits')
x1=cat1['xcenter']
y1=cat1['ycenter']
x2=cat2['xcenter']
y2=cat2['ycenter']
ncat1=len(cat1)
ncat2=len(cat2)
XX=[]
YY=[]
for i in range(ncat2):
    XX.extend((x1-x2[i]))
    YY.extend((y1-y2[i]))
XX=np.array(XX)
YY=np.array(YY)
xhist,xbins=np.histogram(XX,range=[-200,200],bins=401)
yhist,ybins=np.histogram(YY,range=[-200,200],bins=401)
print(np.median(xhist),np.median(yhist))
f,axs=plt.subplots(1,2,figsize=(16,8))
axs[0].hist(XX,range=[-200,200],bins=401)
axs[0].set_title("x shift")
axs[1].hist(YY,range=[-200,200],bins=401)
axs[1].set_title("y shift")


## Apply shift

In [ ]:
## calculate shifts and update catalogs

for i,ifile in enumerate(cfiles):
    rootname,_=os.path.splitext(ifile)
    catfile=rootname+'-cat.fits'
    print("calculate shifts :", i+1,len(cfiles),ifile)
    if i==0:
        cat1=Table.read(catfile)
        x1=cat1['xcenter']
        y1=cat1['ycenter']
        if 'x_sht' not in cat1.colnames:
            xcol=Table.Column(x1,name='x_sht')
            ycol=Table.Column(y1,name='y_sht')
            cat1.add_columns([xcol,ycol])
        else:
            cat1['x_sht']=x1
            cat1['y_sht']=y1
        cat1.write(catfile,overwrite=True)
    else:
        cat2=Table.read(catfile)
        ncat2=len(cat2)
        x2=cat2['xcenter']
        y2=cat2['ycenter']
        XX=[]; YY=[]
        for j in range(ncat2):
            XX.extend((x1-x2[j]))
            YY.extend((y1-y2[j]))
        XX=np.array(XX)
        YY=np.array(YY)
        xhist,xbins=np.histogram(XX,range=[-200,200],bins=401)
        yhist,ybins=np.histogram(YY,range=[-200,200],bins=401)
        idx=np.argmax(xhist)
        xsht0=(xbins[idx]+xbins[idx+1])/2.0
        idx=np.argmax(yhist)
        ysht0=(ybins[idx]+ybins[idx+1])/2.0
        print("initial shift:",xsht0,ysht0)
        mask=(np.abs(XX-xsht0)<3) & (np.abs(YY-ysht0)<3)
        #print(mask.sum())
        xsht1=np.median(XX[mask])
        ysht1=np.median(YY[mask])
        print("finetuned shift:",xsht1,ysht1)
        if 'x_sht' not in cat1.colnames:
            xcol=Table.Column(x2+xsht1,name='x_sht')
            ycol=Table.Column(y2+ysht1,name='y_sht')
            cat2.add_columns([xcol,ycol])
        else:
            cat2['x_sht']=x2+xsht1
            cat2['y_sht']=y2+ysht1
        
        cat2.write(catfile,overwrite=True)

In [ ]:
## define target

data=fits.getdata(reduced_data + 'p4466637UW32R004.fit')
x_targ,y_targ=(193.39-1,358.18-1)
#x_comp,y_comp=(159.54-1,336.61-1)
#x_vali,y_vali=(111.89-1,358.47-1)
x_comp,y_comp=(329.82-1,375.68-1)
x_vali,y_vali=(179.24-1,413.60-1)
aper_targ = CircularAperture((x_targ,y_targ), r=10.)
aper_comp = CircularAperture((x_comp,y_comp), r=10.)
aper_vali = CircularAperture((x_vali,y_vali), r=10.)
plt.figure(figsize=(8,8))
plt.imshow(data, cmap='Greys_r', origin='lower', vmin=300,vmax=600, interpolation='nearest')
aper_targ.plot(color='red', lw=1.5, alpha=0.5)
aper_comp.plot(color='cyan', lw=1.5, alpha=0.5)
aper_vali.plot(color='yellow', lw=1.5, alpha=0.5)
plt.title('red: target, cyan: comparison, yellow: validation')


## Run photometry

In [ ]:
## prepare data

from astropy.time import Time
naper=len(radii)
nfiles=len(cfiles)
lc_targ=np.zeros((1+2*naper,nfiles))
lc_comp=np.zeros((1+2*naper,nfiles))
lc_vali=np.zeros((1+2*naper,nfiles))
print("calculating light curves...")
for i,ifile in enumerate(cfiles):
 rootname,_=os.path.splitext(ifile)
 head=fits.getheader(ifile)
 datestr=head['DATE-OBS']
 date=np.array(datestr.split('/')).astype('int')
 date[2]=2000
 timestr=head['TIME']
 datetime="%4d-%2d-%2d"%(date[2],date[1],date[0])+'T'+timestr.strip()
 t=Time(datetime,format='isot',scale='utc')
 jd=t.mjd
 lc_targ[0,i]=jd
 lc_comp[0,i]=jd
 lc_vali[0,i]=jd
 
 print("MJD: ",datetime,jd)
 catfile=rootname+'-cat.fits'
 print("reading:", i+1,len(cfiles),ifile)
 
 cat=fits.getdata(catfile)
 x=cat['x_sht']
 y=cat['y_sht']
 
 # get target star
 d=np.sqrt((x-x_targ)**2+(y-y_targ)**2)
 idx=np.argmin(d)
 icat=cat[idx]
 dt=d[idx]
 if d[idx]<2:
     for j in range(naper):
         lc_targ[j+1,i]=icat['aperture_sum_'+str(j)]
         lc_targ[naper+j+1,i]=icat['aperture_sum_err_'+str(j)]
 else:
     lc_targ[1:,i]=np.nan
 
 
 # get comparison star
 d=np.sqrt((x-x_comp)**2+(y-y_comp)**2)
 idx=np.argmin(d)
 icat=cat[idx]
 dc=d[idx]
 if d[idx]<2:
     for j in range(naper):
         lc_comp[j+1,i]=icat['aperture_sum_'+str(j)]
         lc_comp[naper+j+1,i]=icat['aperture_sum_err_'+str(j)]
 else:
     lc_comp[1:,i]=np.nan
 
 # get validation star
 d=np.sqrt((x-x_vali)**2+(y-y_vali)**2)
 idx=np.argmin(d)
 icat=cat[idx]
 dv=d[idx]
 if d[idx]<2:
     for j in range(naper):
         lc_vali[j+1,i]=icat['aperture_sum_'+str(j)]
         lc_vali[naper+j+1,i]=icat['aperture_sum_err_'+str(j)]
 else:
     lc_vali[1:,i]=np.nan
 
 print(dt,dc,dv)

## Plot lightcurves

In [ ]:
### function to show curve according to aperture value

def show_lightcurve(change): #(iaper: int = 4) -> None:
    #iaper=4 # for iaper aperture
    iaper=change['new'] # for iaper aperture
    rlc_targ=lc_targ[iaper+1,:]/lc_comp[iaper+1,:]
    rlc_vali=lc_vali[iaper+1,:]/lc_comp[iaper+1,:]
    a1=1.0/lc_comp[iaper+1,:]; e1=lc_targ[iaper+naper+1,:]
    a2=lc_targ[iaper+1,:]/lc_comp[iaper+1,:]**2; e2=lc_comp[iaper+naper+1,:]
    rlcerr_targ=np.sqrt(a1**2*e1**2+a2**2*e2**2)
    a1=1.0/lc_comp[iaper+1,:]; e1=lc_vali[iaper+naper+1,:]
    a2=lc_vali[iaper+1,:]/lc_comp[iaper+1,:]**2; e2=lc_comp[iaper+naper+1,:]
    rlcerr_vali=np.sqrt(a1**2*e1**2+a2**2*e2**2)
    idx=np.argmin(np.abs(lc_targ[0,:]-51888.67))
    norm_targ=np.median(rlc_targ[idx:])
    norm_vali=np.median(rlc_vali[idx:])
    tmpx=[np.min(lc_targ[0,:]),np.max(lc_targ[0,:])]
    plt.figure(figsize=(10,6))
    plt.plot(lc_targ[0,:],rlc_targ/norm_targ,'r.')
    plt.plot(lc_targ[0,:],rlc_vali/norm_vali+0.08,'b.')
    plt.plot(tmpx,[1.0,1.0],'g-',linewidth=2)
    plt.plot(tmpx,[1.08,1.08],'g-',linewidth=2)
    plt.ylim([0.95,1.125])
    plt.xlabel('MJD',fontsize=20)
    plt.ylabel('$\Delta m$')
    plt.title("red: exoplanet transit, blue: validation star")
    plt.show()
    print('photerr for target/comparison:',np.median(rlcerr_targ))
    print('photerr for validation/comparison:',np.median(rlcerr_vali))
    #print(iaper, sigma_clipped_stats(2.5*np.log10(rlc_vali),sigma=3,maxiters=3));


In [ ]:
aper_select = widgets.Dropdown(options=radii, value=4, description='Aperture size:')
display(aper_select)

def handle_aperture_change(change):
    with out:
        out.clear_output()
        show_lightcurve(change)

aper_select.observe(handle_aperture_change, names='value')


In [ ]:
out.clear_output()